In [ ]:
# Library Imports
from transformers import DistilBertTokenizerFast, AutoTokenizer, AutoModelForQuestionAnswering, set_seed
from datasets import *
import numpy as np
import pandas as pd
import torch
import ast
import evaluate

In [ ]:
# Set the seed
SEED = 42
set_seed(SEED)

# Standard model path
local_models_path = '../models/BERT'

# Boilerplate results
results = pd.DataFrame(columns=['Model', 'Train Data', 'Data Type', 'Bert.Precision', 'Bert.Recall', 'Bert.F1', 'BLEU', 'Squad.Exact', 'Squad.F1'])

In [ ]:
# Define the prediction function
def inference_answer(question, context, tokenizer, model):
    question = question
    context = context
    test_feature = tokenizer(
        question,
        context,
        max_length=318
    )
    with torch.no_grad():
        outputs = model(torch.tensor([test_feature["input_ids"]]))
    start_logits = outputs.start_logits.cpu().numpy()
    end_logits = outputs.end_logits.cpu().numpy()
    answer_ids = test_feature["input_ids"][np.argmax(
        start_logits):np.argmax(end_logits)+1]
    return " ".join(tokenizer.batch_decode(answer_ids, skip_special_tokens=True))

## Load all data

2020

In [ ]:
# Load the dataset from file and split it into train and test datasets
data_2020_full = load_dataset('csv', data_files=f"../data/clean/squad/sustainability-report-2020-squad-format.csv",
                    delimiter=";", split='train').train_test_split(test_size=0.3, shuffle=True, seed=SEED)

# Reformat the train and test set such as they adhere to the SQuAD format (reading from cvs loads strings not objects as expected)
data_2020_full["test"] = data_2020_full["test"].map(
    lambda example: ast.literal_eval(example["answers"]))
data_2020_full["test"] = data_2020_full["test"].map(lambda example: {"question": example["question"], "context": example["context"], "answers": {
                                "text": example["text"], "answer_start": example["answer_start"]}})
# Replace all "\n" with " " in the context, answers and questions
data_2020_full["test"] = data_2020_full["test"].map(lambda example: {"question": example["question"].replace("\n", " "), "context": example["context"].replace("\n", " "), "answers": {
                                "text": [example["answers"]["text"][0].replace("\n", " ")], "answer_start": example["answers"]["answer_start"]}})
data_2020_full["test"] = data_2020_full["test"].remove_columns(["text", "answer_start"])
# Get ground truth answers
test_data_2020_full = data_2020_full["test"]
gt_answers_2020_full = [temp["answers"]["text"][0] for temp in test_data_2020_full]

# Squad formatted data
references_2020 = [{"answers": {"answer_start": [answer["answer_start"][0]], "text": [answer["text"][0]]}, "id": str(id)} for id, answer in zip(data_2020_full["test"]["id"], data_2020_full["test"]["answers"])]

2022

In [ ]:
data_2022_full = load_dataset('csv', data_files=f"../data/clean/squad/sustainability-report-2022-squad-format.csv",
                    delimiter=";", split='train').train_test_split(test_size=0.3, shuffle=True, seed=SEED)

# Reformat the train and test set such as they adhere to the SQuAD format (reading from cvs loads strings not objects as expected)
data_2022_full["test"] = data_2022_full["test"].map(
    lambda example: ast.literal_eval(example["answers"]))
data_2022_full["test"] = data_2022_full["test"].map(lambda example: {"question": example["question"], "context": example["context"], "answers": {
                                "text": example["text"], "answer_start": example["answer_start"]}})
# Replace all "\n" with " " in the context, answers and questions
data_2022_full["test"] = data_2022_full["test"].map(lambda example: {"question": example["question"].replace("\n", " "), "context": example["context"].replace("\n", " "), "answers": {
                                "text": [example["answers"]["text"][0].replace("\n", " ")], "answer_start": example["answers"]["answer_start"]}})
data_2022_full["test"] = data_2022_full["test"].remove_columns(["text", "answer_start"])

test_data_2022_full = data_2022_full["test"]
gt_answers_2022_full = [temp["answers"]["text"][0] for temp in test_data_2022_full]

# Squad formatted data
references_2022 = [{"answers": {"answer_start": [answer["answer_start"][0]], "text": [answer["text"][0]]}, "id": str(id)} for id, answer in zip(data_2022_full["test"]["id"], data_2022_full["test"]["answers"])]

2020 + 2022

In [ ]:
data_2020_2022 = load_dataset('csv', data_files="../data/clean/squad/sustainability-report-2042-squad-format.csv",
                                delimiter=";", split="train").train_test_split(test_size=0.3, shuffle=True, seed=SEED)

# Reformat the train and test set such as they adhere to the SQuAD format (reading from cvs loads strings not objects as expected)
data_2020_2022["test"] = data_2020_2022["test"].map(
    lambda example: ast.literal_eval(example["answers"]))
data_2020_2022["test"] = data_2020_2022["test"].map(lambda example: {"question": example["question"], "context": example["context"], "answers": {
                                "text": example["text"], "answer_start": example["answer_start"]}})
# Replace all "\n" with " " in the context, answers and questions
data_2020_2022["test"] = data_2020_2022["test"].map(lambda example: {"question": example["question"].replace("\n", " "), "context": example["context"].replace("\n", " "), "answers": {
                                "text": [example["answers"]["text"][0].replace("\n", " ")], "answer_start": example["answers"]["answer_start"]}})
data_2020_2022["test"] = data_2020_2022["test"].remove_columns(["text", "answer_start"])
# Get ground truth answers
test_data_2020_2022 = data_2020_2022["test"]
gt_answers_2020_2022 = [temp["answers"]["text"][0] for temp in test_data_2020_2022]

# Squad formatted data
references_2020_2022 = [{"answers": {"answer_start": [answer["answer_start"][0]], "text": [answer["text"][0]]}, "id": str(id)} for id, answer in zip(data_2020_2022["test"]["id"], data_2020_2022["test"]["answers"])]

2022 handwritten

In [ ]:
data_2022_handwritten = load_dataset('csv', data_files=f"../data/clean/squad/QA_SR_2022_Expert-squad-format.csv",
                                        delimiter=";", split='train').train_test_split(test_size=0.3, shuffle=True, seed=SEED)

# Reformat the train and test set such as they adhere to the SQuAD format (reading from cvs loads strings not objects as expected)
data_2022_handwritten["test"] = data_2022_handwritten["test"].map(
    lambda example: ast.literal_eval(example["answers"]))
data_2022_handwritten["test"] = data_2022_handwritten["test"].map(lambda example: {"question": example["question"], "context": example["context"], "answers": {
                                "text": example["text"], "answer_start": example["answer_start"]}})
# Replace all "\n" with " " in the context, answers and questions
data_2022_handwritten["test"] = data_2022_handwritten["test"].map(lambda example: {"question": example["question"].replace("\n", " "), "context": example["context"].replace("\n", " "), "answers": {
                                "text": [example["answers"]["text"][0].replace("\n", " ")], "answer_start": example["answers"]["answer_start"]}})
data_2022_handwritten["test"] = data_2022_handwritten["test"].remove_columns(["text", "answer_start"])

test_data_2022_handwritten = data_2022_handwritten["test"]
gt_answers_2022_handwritten = [temp["answers"]["text"][0] for temp in test_data_2022_handwritten]

# Squad formatted data
references_2022_handwritten = [{"answers": {"answer_start": [answer["answer_start"][0]], "text": [answer["text"][0]]}, "id": str(id)} for id, answer in enumerate(data_2022_handwritten["test"]["answers"])]

## SIMPLE EVALUATION

In [ ]:
bertscore = evaluate.load("bertscore")
bleu = evaluate.load("bleu")
squad_v2_metric = evaluate.load("squad_v2")

### Distilbert

In [ ]:
model_name = "distilbert-base-cased-distilled-squad"
tokenizer = DistilBertTokenizerFast.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

answers_2020 = [inference_answer(data_2020_full["test"]["question"][idx], data_2020_full["test"]["context"][idx], tokenizer, model) for idx in range(data_2020_full["test"].shape[0])]
answers_2020_squad = [{"id": str(id), "prediction_text": answer, "no_answer_probability": 0.} for id, answer in zip(data_2020_full["test"]["id"], answers_2020)]
answers_2022 = [inference_answer(data_2022_full["test"]["question"][idx], data_2022_full["test"]["context"][idx], tokenizer, model) for idx in range(data_2022_full["test"].shape[0])]
answers_2022_squad = [{"id": str(id), "prediction_text": answer, "no_answer_probability": 0.} for id, answer in zip(data_2022_full["test"]["id"], answers_2022)]
answers_2020_2022 = [inference_answer(data_2020_2022["test"]["question"][idx], data_2020_2022["test"]["context"][idx], tokenizer, model) for idx in range(data_2020_2022["test"].shape[0])]
answers_2020_2022_squad = [{"id": str(id), "prediction_text": answer, "no_answer_probability": 0.} for id, answer in zip(data_2020_2022["test"]["id"], answers_2020_2022)]
answers_2022_handwritten = [inference_answer(data_2022_handwritten["test"]["question"][idx], data_2022_handwritten["test"]["context"][idx], tokenizer, model) for idx in range(data_2022_handwritten["test"].shape[0])]
answers_2022_handwritten_squad = [{"id": str(id), "prediction_text": answer, "no_answer_probability": 0.} for id, answer in enumerate(answers_2022_handwritten)]

# bertscore
bert_results_2020 = bertscore.compute(predictions=answers_2020, references=gt_answers_2020_full, lang="en")
bert_results_2022 = bertscore.compute(predictions=answers_2022, references=gt_answers_2022_full, lang="en")
bert_results_2020_2022 = bertscore.compute(predictions=answers_2020_2022, references=gt_answers_2020_2022, lang="en")
bert_results_2022_handwritten = bertscore.compute(predictions=answers_2022_handwritten, references=gt_answers_2022_handwritten, lang="en")

# bleu
bleu_results_2020 = bleu.compute(predictions=answers_2020, references=gt_answers_2020_full)
bleu_results_2022 = bleu.compute(predictions=answers_2022, references=gt_answers_2022_full)
bleu_results_2020_2022 = bleu.compute(predictions=answers_2020_2022, references=gt_answers_2020_2022)
bleu_results_2022_handwritten = bleu.compute(predictions=answers_2022_handwritten, references=gt_answers_2022_handwritten)

# squad_v2
squad_results_2020 = squad_v2_metric.compute(predictions=answers_2020_squad, references=references_2020)
squad_results_2022 = squad_v2_metric.compute(predictions=answers_2022_squad, references=references_2022)
squad_results_2020_2022 = squad_v2_metric.compute(predictions=answers_2020_2022_squad, references=references_2020_2022)
squad_results_2022_handwritten = squad_v2_metric.compute(predictions=answers_2022_handwritten_squad, references=references_2022_handwritten)

# add results to dataframe
results.loc[len(results)] = ['distilbert', None, '2020', np.array(bert_results_2020['precision']).mean(), np.array(bert_results_2020['recall']).mean(), np.array(bert_results_2020['f1']).mean(), bleu_results_2020['bleu'], squad_results_2020['exact'], squad_results_2020['f1']]
results.loc[len(results)] = ['distilbert', None, '2022', np.array(bert_results_2022['precision']).mean(), np.array(bert_results_2022['recall']).mean(), np.array(bert_results_2022['f1']).mean(), bleu_results_2022['bleu'], squad_results_2022['exact'], squad_results_2022['f1']]
results.loc[len(results)] = ['distilbert', None, '2020-2022', np.array(bert_results_2020_2022['precision']).mean(), np.array(bert_results_2020_2022['recall']).mean(), np.array(bert_results_2020_2022['f1']).mean(), bleu_results_2020_2022['bleu'], squad_results_2020_2022['exact'], squad_results_2020_2022['f1']]
results.loc[len(results)] = ['distilbert', None, '2022 handwritten', np.array(bert_results_2022_handwritten['precision']).mean(), np.array(bert_results_2022_handwritten['recall']).mean(), np.array(bert_results_2022_handwritten['f1']).mean(), bleu_results_2022_handwritten['bleu'], squad_results_2022_handwritten['exact'], squad_results_2022_handwritten['f1']]

### Roberta

In [ ]:
model_name = "deepset/roberta-base-squad2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

answers_2020 = [inference_answer(data_2020_full["test"]["question"][idx], data_2020_full["test"]["context"][idx], tokenizer, model) for idx in range(data_2020_full["test"].shape[0])]
answers_2020_squad = [{"id": str(id), "prediction_text": answer, "no_answer_probability": 0.} for id, answer in zip(data_2020_full["test"]["id"], answers_2020)]
answers_2022 = [inference_answer(data_2022_full["test"]["question"][idx], data_2022_full["test"]["context"][idx], tokenizer, model) for idx in range(data_2022_full["test"].shape[0])]
answers_2022_squad = [{"id": str(id), "prediction_text": answer, "no_answer_probability": 0.} for id, answer in zip(data_2022_full["test"]["id"], answers_2022)]
answers_2020_2022 = [inference_answer(data_2020_2022["test"]["question"][idx], data_2020_2022["test"]["context"][idx], tokenizer, model) for idx in range(data_2020_2022["test"].shape[0])]
answers_2020_2022_squad = [{"id": str(id), "prediction_text": answer, "no_answer_probability": 0.} for id, answer in zip(data_2020_2022["test"]["id"], answers_2020_2022)]
answers_2022_handwritten = [inference_answer(data_2022_handwritten["test"]["question"][idx], data_2022_handwritten["test"]["context"][idx], tokenizer, model) for idx in range(data_2022_handwritten["test"].shape[0])]
answers_2022_handwritten_squad = [{"id": str(id), "prediction_text": answer, "no_answer_probability": 0.} for id, answer in enumerate(answers_2022_handwritten)]

# bertscore
bert_results_2020 = bertscore.compute(predictions=answers_2020, references=gt_answers_2020_full, lang="en")
bert_results_2022 = bertscore.compute(predictions=answers_2022, references=gt_answers_2022_full, lang="en")
bert_results_2020_2022 = bertscore.compute(predictions=answers_2020_2022, references=gt_answers_2020_2022, lang="en")
bert_results_2022_handwritten = bertscore.compute(predictions=answers_2022_handwritten, references=gt_answers_2022_handwritten, lang="en")

# bleu
bleu_results_2020 = bleu.compute(predictions=answers_2020, references=gt_answers_2020_full)
bleu_results_2022 = bleu.compute(predictions=answers_2022, references=gt_answers_2022_full)
bleu_results_2020_2022 = bleu.compute(predictions=answers_2020_2022, references=gt_answers_2020_2022)
bleu_results_2022_handwritten = bleu.compute(predictions=answers_2022_handwritten, references=gt_answers_2022_handwritten)

# squad_v2
squad_results_2020 = squad_v2_metric.compute(predictions=answers_2020_squad, references=references_2020)
squad_results_2022 = squad_v2_metric.compute(predictions=answers_2022_squad, references=references_2022)
squad_results_2020_2022 = squad_v2_metric.compute(predictions=answers_2020_2022_squad, references=references_2020_2022)
squad_results_2022_handwritten = squad_v2_metric.compute(predictions=answers_2022_handwritten_squad, references=references_2022_handwritten)

# add results to dataframe
results.loc[len(results)] = ['roberta', None, '2020', np.array(bert_results_2020['precision']).mean(), np.array(bert_results_2020['recall']).mean(), np.array(bert_results_2020['f1']).mean(), bleu_results_2020['bleu'], squad_results_2020['exact'], squad_results_2020['f1']]
results.loc[len(results)] = ['roberta', None, '2022', np.array(bert_results_2022['precision']).mean(), np.array(bert_results_2022['recall']).mean(), np.array(bert_results_2022['f1']).mean(), bleu_results_2022['bleu'], squad_results_2022['exact'], squad_results_2022['f1']]
results.loc[len(results)] = ['roberta', None, '2020-2022', np.array(bert_results_2020_2022['precision']).mean(), np.array(bert_results_2020_2022['recall']).mean(), np.array(bert_results_2020_2022['f1']).mean(), bleu_results_2020_2022['bleu'], squad_results_2020_2022['exact'], squad_results_2020_2022['f1']]
results.loc[len(results)] = ['roberta', None, '2022 handwritten', np.array(bert_results_2022_handwritten['precision']).mean(), np.array(bert_results_2022_handwritten['recall']).mean(), np.array(bert_results_2022_handwritten['f1']).mean(), bleu_results_2022_handwritten['bleu'], squad_results_2022_handwritten['exact'], squad_results_2022_handwritten['f1']]

### Distilbert - finetuned

In [ ]:
model_name_2020 = f"{local_models_path}/distilbert-base-cased-distilled-squad-finetuned-NLB-QA-2020-full"
tokenizer_2020 = DistilBertTokenizerFast.from_pretrained(model_name_2020)
model_2020 = AutoModelForQuestionAnswering.from_pretrained(model_name_2020)

model_name_2022 = f"{local_models_path}/distilbert-base-cased-distilled-squad-finetuned-NLB-QA-2022-full"
tokenizer_2022 = DistilBertTokenizerFast.from_pretrained(model_name_2022)
model_2022 = AutoModelForQuestionAnswering.from_pretrained(model_name_2022)

model_name_2020_2022 = f"{local_models_path}/distilbert-base-cased-distilled-squad-finetuned-NLB-QA-2042-full_combined"
tokenizer_2020_2022 = DistilBertTokenizerFast.from_pretrained(model_name_2020_2022)
model_2020_2022 = AutoModelForQuestionAnswering.from_pretrained(model_name_2020_2022)

model_name_2022_handwritten = f"{local_models_path}/distilbert-base-cased-distilled-squad-finetuned-NLB-QA-2022-handwritten"
tokenizer_2022_handwritten = DistilBertTokenizerFast.from_pretrained(model_name_2022_handwritten)
model_2022_handwritten = AutoModelForQuestionAnswering.from_pretrained(model_name_2022_handwritten)

answers_2020 = [inference_answer(data_2020_full["test"]["question"][idx], data_2020_full["test"]["context"][idx], tokenizer_2020, model_2020) for idx in range(data_2020_full["test"].shape[0])]
answers_2020_squad = [{"id": str(id), "prediction_text": answer, "no_answer_probability": 0.} for id, answer in zip(data_2020_full["test"]["id"], answers_2020)]
answers_2022 = [inference_answer(data_2022_full["test"]["question"][idx], data_2022_full["test"]["context"][idx], tokenizer_2022, model_2022) for idx in range(data_2022_full["test"].shape[0])]
answers_2022_squad = [{"id": str(id), "prediction_text": answer, "no_answer_probability": 0.} for id, answer in zip(data_2022_full["test"]["id"], answers_2022)]
answers_2020_2022 = [inference_answer(data_2020_2022["test"]["question"][idx], data_2020_2022["test"]["context"][idx], tokenizer_2020_2022, model_2020_2022) for idx in range(data_2020_2022["test"].shape[0])]
answers_2020_2022_squad = [{"id": str(id), "prediction_text": answer, "no_answer_probability": 0.} for id, answer in zip(data_2020_2022["test"]["id"], answers_2020_2022)]
answers_2022_handwritten = [inference_answer(data_2022_handwritten["test"]["question"][idx], data_2022_handwritten["test"]["context"][idx], tokenizer_2022_handwritten, model_2022_handwritten) for idx in range(data_2022_handwritten["test"].shape[0])]
answers_2022_handwritten_squad = [{"id": str(id), "prediction_text": answer, "no_answer_probability": 0.} for id, answer in enumerate(answers_2022_handwritten)]

# bertscore
bert_results_2020 = bertscore.compute(predictions=answers_2020, references=gt_answers_2020_full, lang="en")
bert_results_2022 = bertscore.compute(predictions=answers_2022, references=gt_answers_2022_full, lang="en")
bert_results_2020_2022 = bertscore.compute(predictions=answers_2020_2022, references=gt_answers_2020_2022, lang="en")
bert_results_2022_handwritten = bertscore.compute(predictions=answers_2022_handwritten, references=gt_answers_2022_handwritten, lang="en")

# bleu
bleu_results_2020 = bleu.compute(predictions=answers_2020, references=gt_answers_2020_full)
bleu_results_2022 = bleu.compute(predictions=answers_2022, references=gt_answers_2022_full)
bleu_results_2020_2022 = bleu.compute(predictions=answers_2020_2022, references=gt_answers_2020_2022)
bleu_results_2022_handwritten = bleu.compute(predictions=answers_2022_handwritten, references=gt_answers_2022_handwritten)

# squad_v2
squad_results_2020 = squad_v2_metric.compute(predictions=answers_2020_squad, references=references_2020)
squad_results_2022 = squad_v2_metric.compute(predictions=answers_2022_squad, references=references_2022)
squad_results_2020_2022 = squad_v2_metric.compute(predictions=answers_2020_2022_squad, references=references_2020_2022)
squad_results_2022_handwritten = squad_v2_metric.compute(predictions=answers_2022_handwritten_squad, references=references_2022_handwritten)

# add results to dataframe
results.loc[len(results)] = ['distilbert', 'full', '2020', np.array(bert_results_2020['precision']).mean(), np.array(bert_results_2020['recall']).mean(), np.array(bert_results_2020['f1']).mean(), bleu_results_2020['bleu'], squad_results_2020['exact'], squad_results_2020['f1']]
results.loc[len(results)] = ['distilbert', 'full', '2022', np.array(bert_results_2022['precision']).mean(), np.array(bert_results_2022['recall']).mean(), np.array(bert_results_2022['f1']).mean(), bleu_results_2022['bleu'], squad_results_2022['exact'], squad_results_2022['f1']]
results.loc[len(results)] = ['distilbert', 'full', '2020-2022', np.array(bert_results_2020_2022['precision']).mean(), np.array(bert_results_2020_2022['recall']).mean(), np.array(bert_results_2020_2022['f1']).mean(), bleu_results_2020_2022['bleu'], squad_results_2020_2022['exact'], squad_results_2020_2022['f1']]
results.loc[len(results)] = ['distilbert', 'full', '2022 handwritten', np.array(bert_results_2022_handwritten['precision']).mean(), np.array(bert_results_2022_handwritten['recall']).mean(), np.array(bert_results_2022_handwritten['f1']).mean(), bleu_results_2022_handwritten['bleu'], squad_results_2022_handwritten['exact'], squad_results_2022_handwritten['f1']]

### Roberta - finetuned

In [ ]:
model_name_2020 = f"{local_models_path}/roberta-base-squad2-finetuned-NLB-QA-2020-full"
tokenizer_2020 = AutoTokenizer.from_pretrained(model_name_2020)
model_2020 = AutoModelForQuestionAnswering.from_pretrained(model_name_2020)

model_name_2022 = f"{local_models_path}/roberta-base-squad2-finetuned-NLB-QA-2022-full"
tokenizer_2022 = AutoTokenizer.from_pretrained(model_name_2022)
model_2022 = AutoModelForQuestionAnswering.from_pretrained(model_name_2022)

model_name_2020_2022 = f"{local_models_path}/roberta-base-squad2-finetuned-NLB-QA-2042-full_combined"
tokenizer_2020_2022 = AutoTokenizer.from_pretrained(model_name_2020_2022)
model_2020_2022 = AutoModelForQuestionAnswering.from_pretrained(model_name_2020_2022)

model_name_2022_handwritten = f"{local_models_path}/roberta-base-squad2-finetuned-NLB-QA-2022-handwritten"
tokenizer_2022_handwritten = AutoTokenizer.from_pretrained(model_name_2022_handwritten)
model_2022_handwritten = AutoModelForQuestionAnswering.from_pretrained(model_name_2022_handwritten)

answers_2020 = [inference_answer(data_2020_full["test"]["question"][idx], data_2020_full["test"]["context"][idx], tokenizer_2020, model_2020) for idx in range(data_2020_full["test"].shape[0])]
answers_2020_squad = [{"id": str(id), "prediction_text": answer, "no_answer_probability": 0.} for id, answer in zip(data_2020_full["test"]["id"], answers_2020)]
answers_2022 = [inference_answer(data_2022_full["test"]["question"][idx], data_2022_full["test"]["context"][idx], tokenizer_2022, model_2022) for idx in range(data_2022_full["test"].shape[0])]
answers_2022_squad = [{"id": str(id), "prediction_text": answer, "no_answer_probability": 0.} for id, answer in zip(data_2022_full["test"]["id"], answers_2022)]
answers_2020_2022 = [inference_answer(data_2020_2022["test"]["question"][idx], data_2020_2022["test"]["context"][idx], tokenizer_2020_2022, model_2020_2022) for idx in range(data_2020_2022["test"].shape[0])]
answers_2020_2022_squad = [{"id": str(id), "prediction_text": answer, "no_answer_probability": 0.} for id, answer in zip(data_2020_2022["test"]["id"], answers_2020_2022)]
answers_2022_handwritten = [inference_answer(data_2022_handwritten["test"]["question"][idx], data_2022_handwritten["test"]["context"][idx], tokenizer_2022_handwritten, model_2022_handwritten) for idx in range(data_2022_handwritten["test"].shape[0])]
answers_2022_handwritten_squad = [{"id": str(id), "prediction_text": answer, "no_answer_probability": 0.} for id, answer in enumerate(answers_2022_handwritten)]

# bertscore
bert_results_2020 = bertscore.compute(predictions=answers_2020, references=gt_answers_2020_full, lang="en")
bert_results_2022 = bertscore.compute(predictions=answers_2022, references=gt_answers_2022_full, lang="en")
bert_results_2020_2022 = bertscore.compute(predictions=answers_2020_2022, references=gt_answers_2020_2022, lang="en")
bert_results_2022_handwritten = bertscore.compute(predictions=answers_2022_handwritten, references=gt_answers_2022_handwritten, lang="en")

# bleu
bleu_results_2020 = bleu.compute(predictions=answers_2020, references=gt_answers_2020_full)
bleu_results_2022 = bleu.compute(predictions=answers_2022, references=gt_answers_2022_full)
bleu_results_2020_2022 = bleu.compute(predictions=answers_2020_2022, references=gt_answers_2020_2022)
bleu_results_2022_handwritten = bleu.compute(predictions=answers_2022_handwritten, references=gt_answers_2022_handwritten)

# squad_v2
squad_results_2020 = squad_v2_metric.compute(predictions=answers_2020_squad, references=references_2020)
squad_results_2022 = squad_v2_metric.compute(predictions=answers_2022_squad, references=references_2022)
squad_results_2020_2022 = squad_v2_metric.compute(predictions=answers_2020_2022_squad, references=references_2020_2022)
squad_results_2022_handwritten = squad_v2_metric.compute(predictions=answers_2022_handwritten_squad, references=references_2022_handwritten)

# add results to dataframe
results.loc[len(results)] = ['roberta', 'full', '2020', np.array(bert_results_2020['precision']).mean(), np.array(bert_results_2020['recall']).mean(), np.array(bert_results_2020['f1']).mean(), bleu_results_2020['bleu'], squad_results_2020['exact'], squad_results_2020['f1']]
results.loc[len(results)] = ['roberta', 'full', '2022', np.array(bert_results_2022['precision']).mean(), np.array(bert_results_2022['recall']).mean(), np.array(bert_results_2022['f1']).mean(), bleu_results_2022['bleu'], squad_results_2022['exact'], squad_results_2022['f1']]
results.loc[len(results)] = ['roberta', 'full', '2020-2022', np.array(bert_results_2020_2022['precision']).mean(), np.array(bert_results_2020_2022['recall']).mean(), np.array(bert_results_2020_2022['f1']).mean(), bleu_results_2020_2022['bleu'], squad_results_2020_2022['exact'], squad_results_2020_2022['f1']]
results.loc[len(results)] = ['roberta', 'full', '2022 handwritten', np.array(bert_results_2022_handwritten['precision']).mean(), np.array(bert_results_2022_handwritten['recall']).mean(), np.array(bert_results_2022_handwritten['f1']).mean(), bleu_results_2022_handwritten['bleu'], squad_results_2022_handwritten['exact'], squad_results_2022_handwritten['f1']]

### Distilbert - finetuned  - train set halved

In [ ]:
model_name_2020 = f"{local_models_path}/distilbert-base-cased-distilled-squad-finetuned-NLB-QA-2020-smaller"
tokenizer_2020 = DistilBertTokenizerFast.from_pretrained(model_name_2020)
model_2020 = AutoModelForQuestionAnswering.from_pretrained(model_name_2020)

model_name_2022 = f"{local_models_path}/distilbert-base-cased-distilled-squad-finetuned-NLB-QA-2022-smaller"
tokenizer_2022 = DistilBertTokenizerFast.from_pretrained(model_name_2022)
model_2022 = AutoModelForQuestionAnswering.from_pretrained(model_name_2022)

model_name_2020_2022 = f"{local_models_path}/distilbert-base-cased-distilled-squad-finetuned-NLB-QA-2042-smaller_combined"
tokenizer_2020_2022 = DistilBertTokenizerFast.from_pretrained(model_name_2020_2022)
model_2020_2022 = AutoModelForQuestionAnswering.from_pretrained(model_name_2020_2022)

answers_2020 = [inference_answer(data_2020_full["test"]["question"][idx], data_2020_full["test"]["context"][idx], tokenizer_2020, model_2020) for idx in range(data_2020_full["test"].shape[0])]
answers_2020_squad = [{"id": str(id), "prediction_text": answer, "no_answer_probability": 0.} for id, answer in zip(data_2020_full["test"]["id"], answers_2020)]
answers_2022 = [inference_answer(data_2022_full["test"]["question"][idx], data_2022_full["test"]["context"][idx], tokenizer_2022, model_2022) for idx in range(data_2022_full["test"].shape[0])]
answers_2022_squad = [{"id": str(id), "prediction_text": answer, "no_answer_probability": 0.} for id, answer in zip(data_2022_full["test"]["id"], answers_2022)]
answers_2020_2022 = [inference_answer(data_2020_2022["test"]["question"][idx], data_2020_2022["test"]["context"][idx], tokenizer_2020_2022, model_2020_2022) for idx in range(data_2020_2022["test"].shape[0])]
answers_2020_2022_squad = [{"id": str(id), "prediction_text": answer, "no_answer_probability": 0.} for id, answer in zip(data_2020_2022["test"]["id"], answers_2020_2022)]

# bertscore
bert_results_2020 = bertscore.compute(predictions=answers_2020, references=gt_answers_2020_full, lang="en")
bert_results_2022 = bertscore.compute(predictions=answers_2022, references=gt_answers_2022_full, lang="en")
bert_results_2020_2022 = bertscore.compute(predictions=answers_2020_2022, references=gt_answers_2020_2022, lang="en")

# bleu
bleu_results_2020 = bleu.compute(predictions=answers_2020, references=gt_answers_2020_full)
bleu_results_2022 = bleu.compute(predictions=answers_2022, references=gt_answers_2022_full)
bleu_results_2020_2022 = bleu.compute(predictions=answers_2020_2022, references=gt_answers_2020_2022)

# squad_v2
squad_results_2020 = squad_v2_metric.compute(predictions=answers_2020_squad, references=references_2020)
squad_results_2022 = squad_v2_metric.compute(predictions=answers_2022_squad, references=references_2022)
squad_results_2020_2022 = squad_v2_metric.compute(predictions=answers_2020_2022_squad, references=references_2020_2022)

# add results to dataframe
results.loc[len(results)] = ['distilbert', 'smaller', '2020', np.array(bert_results_2020['precision']).mean(), np.array(bert_results_2020['recall']).mean(), np.array(bert_results_2020['f1']).mean(), bleu_results_2020['bleu'], squad_results_2020['exact'], squad_results_2020['f1']]
results.loc[len(results)] = ['distilbert', 'smaller', '2022', np.array(bert_results_2022['precision']).mean(), np.array(bert_results_2022['recall']).mean(), np.array(bert_results_2022['f1']).mean(), bleu_results_2022['bleu'], squad_results_2022['exact'], squad_results_2022['f1']]
results.loc[len(results)] = ['distilbert', 'smaller', '2020-2022', np.array(bert_results_2020_2022['precision']).mean(), np.array(bert_results_2020_2022['recall']).mean(), np.array(bert_results_2020_2022['f1']).mean(), bleu_results_2020_2022['bleu'], squad_results_2020_2022['exact'], squad_results_2020_2022['f1']]

### Roberta - finetuned - train set halved

In [ ]:
model_name_2020 = f"{local_models_path}/roberta-base-squad2-finetuned-NLB-QA-2020-smaller"
tokenizer_2020 = AutoTokenizer.from_pretrained(model_name_2020)
model_2020 = AutoModelForQuestionAnswering.from_pretrained(model_name_2020)

model_name_2022 = f"{local_models_path}/roberta-base-squad2-finetuned-NLB-QA-2022-smaller"
tokenizer_2022 = AutoTokenizer.from_pretrained(model_name_2022)
model_2022 = AutoModelForQuestionAnswering.from_pretrained(model_name_2022)

model_name_2020_2022 = f"{local_models_path}/roberta-base-squad2-finetuned-NLB-QA-2042-smaller_combined"
tokenizer_2020_2022 = AutoTokenizer.from_pretrained(model_name_2020_2022)
model_2020_2022 = AutoModelForQuestionAnswering.from_pretrained(model_name_2020_2022)

answers_2020 = [inference_answer(data_2020_full["test"]["question"][idx], data_2020_full["test"]["context"][idx], tokenizer_2020, model_2020) for idx in range(data_2020_full["test"].shape[0])]
answers_2020_squad = [{"id": str(id), "prediction_text": answer, "no_answer_probability": 0.} for id, answer in zip(data_2020_full["test"]["id"], answers_2020)]
answers_2022 = [inference_answer(data_2022_full["test"]["question"][idx], data_2022_full["test"]["context"][idx], tokenizer_2022, model_2022) for idx in range(data_2022_full["test"].shape[0])]
answers_2022_squad = [{"id": str(id), "prediction_text": answer, "no_answer_probability": 0.} for id, answer in zip(data_2022_full["test"]["id"], answers_2022)]
answers_2020_2022 = [inference_answer(data_2020_2022["test"]["question"][idx], data_2020_2022["test"]["context"][idx], tokenizer_2020_2022, model_2020_2022) for idx in range(data_2020_2022["test"].shape[0])]
answers_2020_2022_squad = [{"id": str(id), "prediction_text": answer, "no_answer_probability": 0.} for id, answer in zip(data_2020_2022["test"]["id"], answers_2020_2022)]

# bertscore
bert_results_2020 = bertscore.compute(predictions=answers_2020, references=gt_answers_2020_full, lang="en")
bert_results_2022 = bertscore.compute(predictions=answers_2022, references=gt_answers_2022_full, lang="en")
bert_results_2020_2022 = bertscore.compute(predictions=answers_2020_2022, references=gt_answers_2020_2022, lang="en")

# bleu
bleu_results_2020 = bleu.compute(predictions=answers_2020, references=gt_answers_2020_full)
bleu_results_2022 = bleu.compute(predictions=answers_2022, references=gt_answers_2022_full)
bleu_results_2020_2022 = bleu.compute(predictions=answers_2020_2022, references=gt_answers_2020_2022)

# squad_v2
squad_results_2020 = squad_v2_metric.compute(predictions=answers_2020_squad, references=references_2020)
squad_results_2022 = squad_v2_metric.compute(predictions=answers_2022_squad, references=references_2022)
squad_results_2020_2022 = squad_v2_metric.compute(predictions=answers_2020_2022_squad, references=references_2020_2022)

# add results to dataframe
results.loc[len(results)] = ['roberta', 'smaller', '2020', np.array(bert_results_2020['precision']).mean(), np.array(bert_results_2020['recall']).mean(), np.array(bert_results_2020['f1']).mean(), bleu_results_2020['bleu'], squad_results_2020['exact'], squad_results_2020['f1']]
results.loc[len(results)] = ['roberta', 'smaller', '2022', np.array(bert_results_2022['precision']).mean(), np.array(bert_results_2022['recall']).mean(), np.array(bert_results_2022['f1']).mean(), bleu_results_2022['bleu'], squad_results_2022['exact'], squad_results_2022['f1']]
results.loc[len(results)] = ['roberta', 'smaller', '2020-2022', np.array(bert_results_2020_2022['precision']).mean(), np.array(bert_results_2020_2022['recall']).mean(), np.array(bert_results_2020_2022['f1']).mean(), bleu_results_2020_2022['bleu'], squad_results_2020_2022['exact'], squad_results_2020_2022['f1']]

## Display all results

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(results)